In [1]:
from IPython.display import display
import os
from PIL import Image
import math
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import fitz
from IPython.display import display
from IPython.display import Image as DisplayImage

import fitz  # Import PyMuPDF
from IPython.display import display  # Import display function
from PIL import Image, ImageDraw, ImageFont  # Import necessary classes from PIL
import io  # Import io module for converting bytes

from tqdm import tqdm
import re

In [2]:
dpi = 300
size_ratio = 1.3
y_range = (39, 575)

In [3]:
def draw_bbox(page, bboxs):

    for bbox in bboxs:  # iterate over all tables
        page.draw_rect(bbox,color=fitz.pdfcolor["blue"], width=1.5)
    pix = page.get_pixmap()  # Render page to an image

    # Convert the pixmap to a PIL Image
    img_bytes = pix.tobytes("png")  # Get the image bytes in PNG format
    img = Image.open(io.BytesIO(img_bytes))
    display(img)
    

In [4]:
def save_image(page, page_num, index, prev_text, bbox, xres, yres, width, height, ext, size_ratio=1.3, dpi=300):
    
    x_scale = dpi /xres  # PyMuPDF의 기본 해상도는 72 DPI입니다.
    y_scale = dpi / yres
    matrix = fitz.Matrix(x_scale, y_scale)  # 변환 행렬 생성
    
    # 선택한 bbox 영역의 이미지를 추출 (해상도 조절 적용)
    pix = page.get_pixmap(matrix=matrix, clip=bbox)
    
    # pix = page.get_pixmap(clip=bbox)
    
    # PIL 이미지로 변환
    image = Image.open(io.BytesIO(pix.tobytes()))
    image = image.resize((int(width*size_ratio), int(height*size_ratio)), Image.Resampling.LANCZOS)
    
    
    # 이미지에 텍스트 추가
    draw = ImageDraw.Draw(image)
    # 폰트는 시스템에 따라 경로가 다를 수 있으며, 필요에 따라 수정해야 할 수 있습니다.
    font = ImageFont.truetype("./font/NanumGothicBold.ttf", 16)  # 폰트와 크기 설정
    draw.text((10, 10), prev_text, fill='black', font=font)
    
    # image.save(f"./image/test.{b['ext']}", dpi=(b['xres'], b['yres']))
    img_url = f"./image/{NAMESPACE}/{page_num}_{index}_img.{ext}"
    image.save(img_url)
    return img_url
    
def save_table(page, page_num, index, prev_text, bbox, xres, yres, width, height, ext, size_ratio=1.3, dpi=300, text_rotation=False):
    if text_rotation:
        page.set_rotation(0)
    x_scale = dpi/2 /xres  # PyMuPDF의 기본 해상도는 72 DPI입니다.
    y_scale = dpi/2 / yres
    matrix = fitz.Matrix(x_scale, y_scale)  # 변환 행렬 생성
    
    # 선택한 bbox 영역의 이미지를 추출 (해상도 조절 적용)
    # pix = page.get_pixmap(matrix=matrix, clip=bbox)
    pix = page.get_pixmap(matrix=matrix, clip=bbox)
    
    # pix = page.get_pixmap(clip=bbox)
    
    # PIL 이미지로 변환
    image = Image.open(io.BytesIO(pix.tobytes()))
    # image = image.resize((int(width*size_ratio), int(height*size_ratio)), Image.Resampling.LANCZOS)
    
    
    # 이미지에 텍스트 추가
    draw = ImageDraw.Draw(image)
    # 폰트는 시스템에 따라 경로가 다를 수 있으며, 필요에 따라 수정해야 할 수 있습니다.
    font = ImageFont.truetype("./font/NanumGothicBold.ttf", 16)  # 폰트와 크기 설정
    draw.text((10, 10), prev_text, fill='black', font=font)
    
    # image.save(f"./image/test.{b['ext']}", dpi=(b['xres'], b['yres']))
    
    if text_rotation:
        image = image.rotate(270,expand=True)
        page.set_rotation(90)
    img_url = f"./image/{NAMESPACE}/{page_num}_{index}_table.{ext}"
    image.save(img_url)
    return img_url
        

In [5]:
def get_rotated_bbox(page, rect):
    """
    Adjust the bounding box coordinates for a page that is rotated 90 degrees clockwise.
    
    Args:
    - page: The page object from PyMuPDF.
    - rect: The original bounding box as a fitz.Rect object.
    
    Returns:
    - A fitz.Rect object representing the adjusted bounding box.
    """
    if page.rotation == 90:
        page_width = page.rect.width
        page_height = page.rect.height
        
        # Transform the rectangle coordinates
        new_x0 = rect.y0
        new_y0 = page_width - rect.x1
        new_x1 = rect.y1
        new_y1 = page_width - rect.x0
        
        # Create a new rectangle with the adjusted coordinates
        adjusted_rect = fitz.Rect(new_x0, new_y0, new_x1, new_y1)
        return adjusted_rect
    
    # If no rotation or a different rotation, return the original rectangle
    return rect

def is_text_rotated(page):
    blocks = page.get_text("dict")['blocks']
    if page.find_tables().tables:
        for b in blocks:
            if 'lines' in b:
                for line in b["lines"]:
                    if 'dir' in line:
                        if line['dir'] == (0.0, -1.0):
                            return True
    else:
        return False
    return False

def analyze_page_layout(page, text_rotate):
    tbls = page.find_tables().tables
    is_two_column = True
    blocks = page.get_text("dict")["blocks"]

    # 텍스트가 로테이트 되어있거나,
    if text_rotate:
        is_two_column = False
    # 테이블이 있는데 width가 페이지 넓이 절반보다 크거나,    
    tbls = page.find_tables().tables
    if tbls:
        for tbl in tbls:
            bbox = tbl.bbox
            if (bbox[2] - bbox[0]) > page.rect.width/2:
                is_two_column = False
    # Block 있는데 width가 페이지 넓이 절반보다 크거나.
    if len(list(filter(lambda x: (x['bbox'][2] - x['bbox'][0])>(page.rect.width/2), blocks)))>0:
        is_two_column = False
    return is_two_column
    
def sort_blocks(blocks, is_two_column, page_mid, page_rotation=False):
    blocks = [
        b for b in blocks 
        if b['bbox'][1] >= y_range[0] and b['bbox'][3] <= y_range[1]
    ]

    if is_two_column:
        left_blocks = [b for b in blocks if b['bbox'][0] < page_mid]
        right_blocks = [b for b in blocks if b['bbox'][0] >= page_mid]
        sorted_blocks = sorted(left_blocks, key=lambda b: (b['bbox'][1], b['bbox'][0])) + \
                        sorted(right_blocks, key=lambda b: (b['bbox'][1], b['bbox'][0]))
    else:
        if page_rotation:
            sorted_blocks = sorted(blocks, key=lambda b: (b['bbox'][0], b['bbox'][1]))
        else:
            sorted_blocks = sorted(blocks, key=lambda b: (b['bbox'][1], b['bbox'][0]))
    return sorted_blocks


def get_tbl_df(tbl, text_rotation=False):
    # 여기서 병합된 셀이 단순하다면 이전 cell ffill customizing 필요
    df = tbl
    df.columns = df.columns = [re.sub(r'^\d+-', '', col) for col in df.columns]
    null_sum = df.isnull().sum()
    null_cols = null_sum[null_sum == df.shape[0]].index
    df = df.drop(null_cols, axis=1)
    if text_rotation:
        
        first_col_as_df = pd.DataFrame([df.columns.tolist(), df.iloc[0,:].values], columns=range(0, len(df.columns)))
        
        empty_str_cols = np.where(first_col_as_df.iloc[1, :].values == '')[0].tolist()
        first_col_as_df.loc[0, empty_str_cols] = ''
        
        None_str_cols = np.where(first_col_as_df.iloc[1, :].values is None)[0].tolist()
        first_col_as_df.loc[0, None_str_cols] = None
        first_col_as_df = first_col_as_df.loc[[0],:]
        
        df.columns = range(0, len(df.columns))
        df = pd.concat([first_col_as_df, df]).fillna(method='ffill')
        df = df.set_index(df.columns[0])
        df = df.T
        df = df.iloc[:,list(range(len(df.columns)-1,-1, -1))]

    df_cols = df.columns.tolist()
    for i in range(1, len(df_cols)):
        if "Col" in df_cols[i]:
            df_cols[i] = df_cols[i-1]  # "COl"을 이전 인덱스의 값으로 대체합니다.
    df.columns = df_cols
    df_md = df.to_markdown()
    return df_md
        

In [6]:
def is_overlapping(area1, area2):
    """
    두 영역이 겹치는지 확인하는 함수.
    area1, area2: (x0, y0, x1, y1) 형식의 영역
    """
    x0_1, y0_1, x1_1, y1_1 = area1
    x0_2, y0_2, x1_2, y1_2 = area2
    return not (x1_1 < x0_2 or x1_2 < x0_1 or y1_1 < y0_2 or y1_2 < y0_1)


In [7]:
def display_pages_with_bboxes_and_captions(page_num, display_pages_with_bboxes_and_captions=True):
    spans_list = []
    contents_list = []
    image_url_list = []
    contents_type = []
    block_list = []
    dpi = 300
    size_ratio = 1.3
    prev_text = ""    

    doc = fitz.open(FILE_NM)
    doc2 = fitz.open(FILE_NM)
    
    page = doc.load_page(page_num)
    draw_page = doc2.load_page(page_num)
    text_rotation = is_text_rotated(page)

    table_areas_on_page = {}
    tbls = []
    for i, tbl in enumerate(page.find_tables().tables):
        table_areas_on_page[i] = get_rotated_bbox(page, fitz.Rect(tbl.bbox))
        tbls.append(tbl.to_pandas())
    

    if text_rotation:
        page.set_rotation(90)
        draw_page.set_rotation(90)
    is_two_column = analyze_page_layout(page, text_rotation)
    blocks = page.get_text("dict")["blocks"]  # Get text blocks
    blocks_sorted = sort_blocks(blocks, is_two_column, page.rect.width / 2, page.rotation)

    r_tbls = []
    for i, tbl in enumerate(page.find_tables().tables):
        r_tbls.append(tbl)

    prev_overlap_tbl_num=-1
    for i, b in enumerate(blocks_sorted):
        overlapping=False
        block_num = f"{page_num}_{i}"
        bbox = b['bbox']  # Get the bounding box of the block
        caption = str(i + 1)
        if b['type'] == 0:  # Block contains text
            # 테이블 bbox안의 텍스트 block이라면 PASS
            block_area = b['bbox']  # 텍스트 블록의 영역 (x0, y0, x1, y1)
            
            for tbl_num, table_area in table_areas_on_page.items():
                if is_overlapping(block_area, table_area):
                    overlapping=True
                    if tbl_num != prev_overlap_tbl_num:
                        # print("TODO: PANDAS EXTACT, SAVE IMAGE")
                        spans_list.append(None)
                        content = get_tbl_df(tbls[tbl_num], text_rotation)
                        contents_list.append(content)
                        # Save Table Image
                        rect = fitz.Rect(r_tbls[tbl_num].bbox)
                        tab_bbox = get_rotated_bbox(page, rect)
                        # draw_page.draw_rect(tab_bbox,color=fitz.pdfcolor["blue"], width=1.5)
                        
                        img_url = save_table(draw_page, page_num, tbl_num, '', tab_bbox, 99, 99, tab_bbox[3]-tab_bbox[1], tab_bbox[2] - tab_bbox[0], 
                                   'jpeg', size_ratio=1.3, dpi=300, text_rotation=text_rotation)

                        
                        image_url_list.append(img_url)
                        block_list.append(block_num)
                        contents_type.append('table')
                        prev_overlap_tbl_num = tbl_num
                        break
                    else:
                        # print("테이블 추출 이미 완료")
                        break
        
                    
            if overlapping:
                continue

            # TODO: INSERT ROW BY LINES
            
            if 'lines' in b:
                for line in b['lines']:
                    line_text = ''.join(list(map(lambda x: x.get('text', ''), line['spans'])))
                
                    line_spans = str(set(list(map(lambda x: str((x.get('font'), x.get('color'), int(x.get('size')))), 
                                                  line['spans'])))).replace('{', '').replace('}', '')                    

                    spans_list.append(line_spans)
                    contents_list.append(line_text)
                    image_url_list.append(None)
                    contents_type.append('text')
                    block_list.append(block_num)
            
                
            # Insert text on the pixmap
              # Reading order number as caption
            rect = fitz.Rect(bbox)  # Create a rectangle
            
            draw_page.insert_text(  # insert footer 50 points above page bottom
                    (bbox[0], bbox[1]), caption, color=(1,0,0)
                )

            #이미지 캡션을 위해 한줄짜리 block인지 확인 후 저장
            text_lines = [span['text'] for line in b["lines"] for span in line["spans"]]
            text_lines = list(filter(lambda x: (x != '̰') and (x != ' ') and (x != '•'),text_lines))
            if len(text_lines) == 1:
                
                spans = b["lines"][0]["spans"][0]
                if not ((spans['size']==6) and (spans['font']=='HyundaiSansTextKR') and 
                        (spans['color']==16777215)):
                    prev_text = text_lines[0]
        if b['type'] != 0:  # Block contains text
            block_area = b['bbox']
            for tbl_num, table_area in table_areas_on_page.items():
                if is_overlapping(block_area, table_area):
                    overlapping=True
                    break
            if overlapping:
                continue
            else:
                # Save Images
                xres = b['xres']  # PyMuPDF의 기본 해상도는 72 DPI입니다.
                yres = b['yres']
                ext = b['ext']
                width = b['width']
                height = b['height']
                
                img_url = save_image(draw_page, page_num, i, prev_text, bbox, xres, yres, width, height, ext, size_ratio=1.3, dpi=300)
                spans_list.append(None)
                contents_list.append(None)
                image_url_list.append(img_url)
                contents_type.append('image')
                block_list.append(block_num)
                prev_text = ""  # 이전 텍스트 초기화
                
    pix = draw_page.get_pixmap()  # Render page to an image

    # Convert the pixmap to a PIL Image
    img_bytes = pix.tobytes("png")  # Get the image bytes in PNG format
    img = Image.open(io.BytesIO(img_bytes))

    # Display the image in the notebook
    if display_pages_with_bboxes_and_captions:
        display(img)
    # return blocks_sorted
    return {'span': spans_list, 'contents': contents_list, 'img_urls': image_url_list, 'contents_type': contents_type, 'block_num': block_list}

In [8]:
# FILE_NM = "2023 더 뉴 아반떼 - 정기점검.pdf"#"2024 아이오닉 5 (IONIQ 5) NE.pdf"
FILE_NM = "./pdf/2024 아이오닉 5 (IONIQ 5) NE.pdf"

# NAMESPACE = 'the_new_avante_2023'#'ioniq5_2024'
NAMESPACE = 'ioniq5_2024'

doc = fitz.open(FILE_NM)
os.makedirs(f'./image/{NAMESPACE}', exist_ok=True)

# Raw Parsing(텍스트, 이미지, 표)

In [ ]:
span_list = []
contents_list = []
img_urls = []
contents_type = []
block_nums = []

for page_num in tqdm(range(7, len(doc))):
    v = display_pages_with_bboxes_and_captions(page_num, display_pages_with_bboxes_and_captions=False)
    span_list.extend(v['span'])
    contents_list.extend(v['contents'])
    img_urls.extend(v['img_urls'])
    contents_type.extend(v['contents_type'])
    block_nums.extend(v['block_num'])

In [17]:

ioniq_df = pd.DataFrame({
    'span': span_list,
    'contents': contents_list,
    'img_urls': img_urls,
    'ctype': contents_type,
    'block_num': block_nums
})

In [18]:
ioniq_df.to_csv('./parse_result/raw_ioniq5_result.csv', index=False)
ioniq_df.to_parquet('./parse_result/raw_ioniq5_result.parquet', index=False)

In [ ]:
delete_mask = (pc['contents_meta'] == 'h1') | (pc['contents_meta'] == 'h2')
pc['delete_mask'] = delete_mask
pc['delete_mask_cumsum'] = delete_mask.cumsum()
cumsum = delete_mask.cumsum()
# 홀수 cumsum은 'h1'과 'h2' 사이를 나타냅니다. 'h1'에서 시작하여 'h2' 이전까지 홀수 값을 갖습니다.
filtered_cumsum = cumsum % 2 == 1

# 'h1'과 'h2' 사이 (하지만 'h2'는 포함)의 row만 선택하여 삭제
# 'h1'은 홀수 cumsum을 가지고 시작되므로 제외되며, 'h2'는 포함시키기 위해 delete_mask를 사용하여 추가합니다.
pc = pc[~filtered_cumsum | delete_mask].reset_index(drop=True)


# 병합

In [130]:
ioniq_df = pd.read_parquet('./parse_result/raw_ioniq5_result.parquet')

In [131]:
len_by_span = ioniq_df.groupby('span').size().sort_values()

In [132]:
ioniq_df_bak = ioniq_df.copy()
ioniq_df = ioniq_df_bak.copy()

## 불필요한 span 조합 텍스트 제거
- 제조사별 일반화 가능할까?

In [133]:
for del_span_type_num in [1,2,3,4,5,6,8,10,11,13,21,27,29,30]:
    ioniq_df = ioniq_df[ioniq_df['span']!=len_by_span.index[del_span_type_num]]

### .......... 인덱스 텍스트 제거

In [134]:
ioniq_df = ioniq_df[~ioniq_df['contents'].astype(str).str.contains('\.\.\.')]

### 빈 표 제거

In [135]:
ioniq_df = ioniq_df[~((ioniq_df['ctype']=='table') & (ioniq_df['contents']==''))]

### Header 생성

In [136]:
ioniq_df['index'] = None

In [137]:
h1_index = 9
ioniq_df.loc[ioniq_df['span']==len_by_span.index[h1_index], 'index'] = 'h1'

In [138]:
h2_index = 18
ioniq_df.loc[ioniq_df['span']==len_by_span.index[h2_index], 'index'] = 'h2'

In [139]:
h3_index = 24
ioniq_df.loc[ioniq_df['span']==len_by_span.index[h3_index], 'index'] = 'h3'

In [140]:
h4_index = 28
ioniq_df.loc[ioniq_df['span']==len_by_span.index[h4_index], 'index'] = 'h4'

### [임시] 첫페이지를 빼서 0번 Header 추가

In [141]:
tmp_df = pd.DataFrame({'span': ['\"(\'HyundaiSansHeadKR\', 13416, 28)\"'],'contents': ['전기차 시작하기'], 'img_urls': [None], 'ctype': 'text', 'index':'h1', 'block_num':'7_0'})
ioniq_df = pd.concat([tmp_df, ioniq_df])
ioniq_df = ioniq_df.reset_index(drop=True)

### 저장 안되는 문자 제거

In [142]:
ioniq_df['contents'] = ioniq_df['contents'].str.replace('\u2022', '')
ioniq_df['contents'] = ioniq_df['contents'].str.replace('\u09a1', '')

### Block 별 병합 전 데이터 저장

In [143]:
ioniq_df.to_csv('./parse_result/line_ioniq5_result.csv', index=False, encoding='cp949')
ioniq_df.to_parquet('./parse_result/line_ioniq5_result.parquet', index=False)

# Block 별 텍스트 병합

In [144]:
ioniq_df = pd.read_parquet('./parse_result/line_ioniq5_result.parquet')

## PDF 내 같은 Block이었던 텍스트끼리 Line 병합

In [145]:
merge_contents = pd.DataFrame(ioniq_df[~ioniq_df['contents'].isnull()].groupby(['block_num'])['contents'].apply(lambda x: ''.join(x))).reset_index()
merge_contents.columns = ['block_num', 'merge_contents']

### 기존 Line에 merge 후 중복 제거

In [146]:
ioniq_df = ioniq_df.merge(merge_contents, on=['block_num'], how='left')
m_ioniq_df = ioniq_df.drop_duplicates('block_num', keep='first').reset_index(drop=True).drop('contents', axis=1)

### Index별로 document 분리

**Header level, Header 내용이 달라질때 마다 document grouping**
- Header: h1, h2, h3, h4


In [147]:
m_ioniq_df['doc_group'] = (~m_ioniq_df['index'].isnull()).cumsum()

In [148]:
m_ioniq_df['h1'] = None
m_ioniq_df['h2'] = None
m_ioniq_df['h3'] = None

In [149]:
m_ioniq_df.loc[m_ioniq_df['index']=='h1', 'h1'] = m_ioniq_df.loc[m_ioniq_df['index']=='h1', 'merge_contents']
m_ioniq_df.loc[m_ioniq_df['index']=='h2', 'h2'] = m_ioniq_df.loc[m_ioniq_df['index']=='h2', 'merge_contents']
m_ioniq_df.loc[m_ioniq_df['index']=='h3', 'h3'] = m_ioniq_df.loc[m_ioniq_df['index']=='h3', 'merge_contents']

- h1이 있는데 h2가 없으면 h2는 empty: h1 바로 밑에 text가 있는 경우
- h2가 있는데 h3가 없으면 h3는 empty: h2 바로 밑에 text가 있는 경우

In [150]:
m_ioniq_df.loc[(~m_ioniq_df['h1'].isna()) & (m_ioniq_df['h2'].isna()), 'h2'] = ''
m_ioniq_df.loc[(~m_ioniq_df['h2'].isna()) & (m_ioniq_df['h3'].isna()), 'h3'] = ''

In [151]:
m_ioniq_df[['h1', 'h2', 'h3']] = m_ioniq_df[['h1', 'h2', 'h3']].fillna(method='ffill')

In [152]:
m_ioniq_df.to_csv('./parse_result/block_ioniq5_result.csv', index=False, encoding='cp949')
m_ioniq_df.to_parquet('./parse_result/block_ioniq5_result.parquet', index=False)

# Doc 병합(H3 레벨)

In [10]:
m_ioniq_df = pd.read_parquet('./parse_result/block_ioniq5_result.parquet')

## 최소 h3 단위로 text 병합

In [11]:
doc_merge_contents = m_ioniq_df[m_ioniq_df['ctype']=='text'].groupby('doc_group')['merge_contents'].apply(lambda x: '\n'.join(x)).reset_index()
doc_merge_contents.columns = ['doc_group', 'doc_contents']

In [12]:
doc_ioniq_df = m_ioniq_df[m_ioniq_df['ctype']=='text'].merge(doc_merge_contents, on='doc_group', how='left').drop_duplicates('doc_group', keep='first')

## Doc 내 이미지 URL 마스킹

In [13]:
doc_images = m_ioniq_df[m_ioniq_df['ctype']=='image']
doc_images = doc_images.groupby('doc_group')['img_urls'].apply(lambda x: list(x)).reset_index()

## Doc 내 테이블 URL 마스킹

In [14]:
doc_tables = m_ioniq_df[m_ioniq_df['ctype']=='table']
doc_tables = doc_tables[doc_tables['merge_contents'] != ''].copy()

doc_table_images = doc_tables.groupby('doc_group')['img_urls'].apply(lambda x: list(x)).reset_index()
doc_tables['merge_contents'] = '표: ' + doc_tables['block_num'] + '\n' + doc_tables['merge_contents'] + '\n' 
doc_tables = doc_tables[['doc_group', 'merge_contents']].merge(doc_table_images, on='doc_group', how='outer')
doc_tables.columns = ['doc_group', 'table_contents', 'table_img_urls']

## Doc 단위 병합

In [15]:
final_doc_df = doc_ioniq_df.drop(['img_urls', 'merge_contents'], axis=1).merge(doc_images, on='doc_group', how='left')\
    .merge(doc_tables, on='doc_group', how='left')
final_doc_df['doc_table_contents'] = ''

In [16]:
final_doc_df.loc[~final_doc_df['table_contents'].isna(), 'doc_table_contents'] = \
    final_doc_df.loc[~final_doc_df['table_contents'].isna(), 'doc_contents'] + '\n####### 표 #######\n' + \
    final_doc_df.loc[~final_doc_df['table_contents'].isna(), 'table_contents'] + '####### 표 종료 #######\n'

final_doc_df.loc[final_doc_df['table_contents'].isna(), 'doc_table_contents'] = \
    final_doc_df.loc[final_doc_df['table_contents'].isna(), 'doc_contents']

# final_doc_df['doc_table_contents'] = final_doc_df['doc_contents'] + '####### 표 #######\n' + final_doc_df['table_contents']

In [17]:
final_doc_df = doc_ioniq_df.drop(['img_urls', 'merge_contents'], axis=1).merge(doc_images, on='doc_group', how='left')\
    .merge(doc_tables, on='doc_group', how='left')
final_doc_df['doc_table_contents'] = ''

In [18]:
final_doc_df.loc[~final_doc_df['table_contents'].isna(), 'doc_table_contents'] = \
    final_doc_df.loc[~final_doc_df['table_contents'].isna(), 'doc_contents'] + '\n####### 표 #######\n' + \
    final_doc_df.loc[~final_doc_df['table_contents'].isna(), 'table_contents'] + '####### 표 종료 #######\n'

final_doc_df.loc[final_doc_df['table_contents'].isna(), 'doc_table_contents'] = \
    final_doc_df.loc[final_doc_df['table_contents'].isna(), 'doc_contents']

# final_doc_df['doc_table_contents'] = final_doc_df['doc_contents'] + '####### 표 #######\n' + final_doc_df['table_contents']

In [19]:
final_doc_df = final_doc_df.drop_duplicates('doc_group')
final_doc_df = final_doc_df.loc[:len(final_doc_df)-2].reset_index(drop=True)

## Doc 단위 저장

In [20]:
final_doc_df.head()

,span,ctype,index,block_num,doc_group,h1,h2,h3,doc_contents,img_urls,table_contents,table_img_urls,doc_table_contents
0,"""('HyundaiSansHeadKR', 13416, 28)""",text,h1,7_0,1,전기차 시작하기,,,전기차 시작하기,NaN,NaN,NaN,전기차 시작하기
1,"""('HyundaiSansTextKRMedium', 13416, 14)""",text,h2,8_0,2,전기차 시작하기,전기 자동차 개요,,전기 자동차 개요\n전기 자동차는 배터리와 전기 모터로 구동되는자동차입니다. 일반 ...,NaN,NaN,NaN,전기 자동차 개요\n전기 자동차는 배터리와 전기 모터로 구동되는자동차입니다. 일반 ...
2,"""('HyundaiSansTextKRMedium', 13416, 12)""",text,h3,8_2,3,전기차 시작하기,전기 자동차 개요,전기 자동차 특징,전기 자동차 특징\n1. 구동용(고전압) 배터리에 충전된 전기를 이용하여 주행하기 ...,NaN,NaN,NaN,전기 자동차 특징\n1. 구동용(고전압) 배터리에 충전된 전기를 이용하여 주행하기 ...
3,"""('HyundaiSansTextKRMedium', 13416, 12)""",text,h3,8_7,4,전기차 시작하기,전기 자동차 개요,배터리에 대한 정보,배터리에 대한 정보\n 본 차량의 배터리는 모터 및 에어컨을 작동시키는 구동용(고전...,NaN,NaN,NaN,배터리에 대한 정보\n 본 차량의 배터리는 모터 및 에어컨을 작동시키는 구동용(고전...
4,"""('HyundaiSansTextKRMedium', 13416, 12)""",text,h3,8_9,5,전기차 시작하기,전기 자동차 개요,전기 자동차의 주요 장치,전기 자동차의 주요 장치\n 구동용(고전압) 배터리 충전장치(OBC): 전력망의 A...,NaN,NaN,NaN,전기 자동차의 주요 장치\n 구동용(고전압) 배터리 충전장치(OBC): 전력망의 A...


In [21]:
final_doc_df.to_csv('./parse_result/doc_ioniq5_result.csv', encoding='cp949')
final_doc_df.to_parquet('./parse_result/doc_ioniq5_result.parquet')